<a href="https://colab.research.google.com/github/Abhilash11Addanki/DeepLearning/blob/main/Custom%20Layers/Custom_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Using Lambda Layer**

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [4]:
x_train,x_test = x_train/255.0,x_test/255.0

In [5]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(128),
                                    tf.keras.layers.Lambda(lambda x:tf.abs(x)),
                                    tf.keras.layers.Dense(10,activation='softmax')])

In [6]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [7]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs = 5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3943 - accuracy: 0.8908 - val_loss: 0.1146 - val_accuracy: 0.9648
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0926 - accuracy: 0.9728 - val_loss: 0.0859 - val_accuracy: 0.9735
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0595 - accuracy: 0.9815 - val_loss: 0.0902 - val_accuracy: 0.9718
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0454 - accuracy: 0.9861 - val_loss: 0.0912 - val_accuracy: 0.9719
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0324 - accuracy: 0.9902 - val_loss: 0.0897 - val_accuracy: 0.9755


**Using Custom Activation Function**

In [8]:
import tensorflow.keras.backend as K

In [9]:
def my_relu(x):
  return K.maximum(0.0,x)

In [10]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(128),
                                    tf.keras.layers.Lambda(my_relu),
                                    tf.keras.layers.Dense(10,activation='softmax')])

In [11]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [12]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs = 5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4387 - accuracy: 0.8781 - val_loss: 0.1551 - val_accuracy: 0.9517
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1201 - accuracy: 0.9641 - val_loss: 0.0955 - val_accuracy: 0.9704
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0759 - accuracy: 0.9771 - val_loss: 0.0805 - val_accuracy: 0.9742
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0551 - accuracy: 0.9828 - val_loss: 0.0843 - val_accuracy: 0.9749
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0441 - accuracy: 0.9869 - val_loss: 0.0816 - val_accuracy: 0.9756


**Creating a Custom Dense Layer**

In [13]:
class SimpleDense(tf.keras.layers.Layer):
  def __init__(self,units=32):
    super(SimpleDense,self).__init__()
    self.units = units
  
  def build(self,input_shape):
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(name='kernel',initial_value=w_init(shape=(input_shape[-1],self.units),dtype='float32'),trainable=True)
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(name='bias',initial_value=b_init(shape=(self.units,),dtype='float32'),trainable=True)

  def call(self,inputs):
    return tf.matmul(inputs,self.w)+self.b

In [17]:
xs = np.array([-1.0,0.0,1.0,2.0,3.0,4.0],dtype=float)
ys = np.array([-3.0,-1.0,1.0,3.0,5.0,7.0],dtype=float)

In [15]:
my_layer = SimpleDense(units=1)
model = tf.keras.models.Sequential([my_layer])

In [16]:
model.compile(optimizer='sgd',loss='mean_squared_error',metrics=['accuracy'])

In [18]:
model.fit(xs,ys,epochs = 500,verbose = 0)
print(model.predict([10.0]))

[[18.981396]]


In [19]:
print(my_layer.variables)

[<tf.Variable 'simple_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9973037]], dtype=float32)>, <tf.Variable 'simple_dense/bias:0' shape=(1,) dtype=float32, numpy=array([-0.99164104], dtype=float32)>]


**Creating a Custom Dense Layer with Activation Function**

In [22]:
class CustomDense(tf.keras.layers.Layer):
  def __init__(self,units=32,activation=None):
    super(CustomDense,self).__init__()
    self.units = units
    self.activation = tf.keras.activations.get(activation)
  
  def build(self,input_shape):
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(name='kernel',initial_value=w_init(shape=(input_shape[-1],self.units),dtype='float32'),trainable=True)
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(name='bias',initial_value=b_init(shape=(self.units,),dtype='float32'),trainable=True)

  def call(self,inputs):
    return self.activation(tf.matmul(inputs,self.w)+self.b)

In [23]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    CustomDense(128,activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(10,activation='softmax')])

In [24]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [25]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs = 5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4758 - accuracy: 0.8606 - val_loss: 0.1353 - val_accuracy: 0.9604
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1488 - accuracy: 0.9551 - val_loss: 0.0957 - val_accuracy: 0.9709
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1073 - accuracy: 0.9671 - val_loss: 0.0858 - val_accuracy: 0.9744
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0831 - accuracy: 0.9743 - val_loss: 0.0866 - val_accuracy: 0.9734
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0725 - accuracy: 0.9778 - val_loss: 0.0754 - val_accuracy: 0.9769
